In [0]:
# Replace these with your actual credentials
ACCESS_KEY = "Access_Key"
SECRET_KEY = "Secret_Key"
REGION = "ap-southeast-2"  # or your actual region



In [0]:
# Configure S3 access
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", ACCESS_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", f"s3.{REGION}.amazonaws.com")

In [0]:
csv_path = "s3a://dj-databricks-pipeline/owid-covid-data.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(csv_path)

df.show(5)
df.printSchema()


+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+------------------------------+-------------------------------

In [0]:
display(df.limit(5).toPandas()) #Since we are not able to read the data properly above now we will use toPandas functions to see the Data

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,null,null,null,null,0.126,0.126,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.33,3.9835428E7,54.422,18.6,2.581,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5,64.83,0.511,null,null,null,null
AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,null,null,null,null,0.126,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.33,3.9835428E7,54.422,18.6,2.581,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5,64.83,0.511,null,null,null,null
AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,null,null,null,null,0.126,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.33,3.9835428E7,54.422,18.6,2.581,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5,64.83,0.511,null,null,null,null
AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,null,null,null,null,0.126,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.33,3.9835428E7,54.422,18.6,2.581,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5,64.83,0.511,null,null,null,null
AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,null,null,null,null,0.126,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,8.33,3.9835428E7,54.422,18.6,2.581,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5,64.83,0.511,null,null,null,null


In [0]:
df.select("iso_code").distinct().count() #Let us check the count of all unique values present in the iso_code

Out[8]: 238

In [0]:
df.groupBy("iso_code").count().show() #TWe will be seeing the count of each country.

+--------+-----+
|iso_code|count|
+--------+-----+
|     BRB|  719|
|     BRA|  739|
|     ARM|  762|
|     ABW|  723|
|     BRN|  727|
|     BOL|  725|
|     ATG|  723|
|OWID_AFR|  752|
|     BHS|  720|
|     BMU|  717|
|     AZE|  735|
|     AUS|  770|
|     AFG|  741|
|     BGD|  733|
|     BLR|  737|
|     BWA|  706|
|     ALB|  740|
|     BHR|  741|
|     BIH|  731|
|     AND|  734|
+--------+-----+
only showing top 20 rows



In [0]:
#LEt us see the count of the continents

df.select("continent").distinct().count()

Out[13]: 7

We do have the data for all the 7 continents.

In [0]:
#Now let us see the count of each continents - how many values are there for each of them.

df.groupBy("continent").count().show()

+-------------+-----+
|    continent|count|
+-------------+-----+
|       Europe|37088|
|       Africa|39417|
|         null| 9956|
|North America|24931|
|South America| 9517|
|      Oceania| 9217|
|         Asia|36200|
+-------------+-----+



In [0]:
#Importing the important libraries
from pyspark.sql.functions import col

#Let us try to change the name of column to something appropriate and keep them sorted by contient name


df.groupBy("continent")\
    .count()\
    .withColumnRenamed("count","countOfRecords")\
    .orderBy(col("continent"))\
    .show()

+-------------+--------------+
|    continent|countOfRecords|
+-------------+--------------+
|         null|          9956|
|       Africa|         39417|
|         Asia|         36200|
|       Europe|         37088|
|North America|         24931|
|      Oceania|          9217|
|South America|          9517|
+-------------+--------------+



This shows that most of the records belong to Africa, Asia and Europe with count of more than 30000+

In [0]:
#But here we are seeing the null values as well. Let us remove them and see the result.
df.filter(col("continent").isNotNull())\
  .groupBy("continent").count().withColumnRenamed("count","countOfRecords")\
    .orderBy(col("continent")).show()

+-------------+--------------+
|    continent|countOfRecords|
+-------------+--------------+
|       Africa|         39417|
|         Asia|         36200|
|       Europe|         37088|
|North America|         24931|
|      Oceania|          9217|
|South America|          9517|
+-------------+--------------+



This shows the count for each column without giving the count of NULL values. 

In [0]:
df.filter(col("continent").isNull()) \
  .select("location") \
  .distinct().show()

+-------------------+
|           location|
+-------------------+
|             Africa|
|               Asia|
|             Europe|
|     European Union|
|        High income|
|Lower middle income|
|      International|
|         Low income|
|      North America|
|            Oceania|
|      South America|
|              World|
|Upper middle income|
+-------------------+



In [0]:
string_cols = [f.name for f in df.schema.fields if f.dataType.simpleString() == 'string'] #Getting all the columns having string data types
numeric_cols = [f.name for f in df.schema.fields if f.dataType.simpleString() in ['double', 'int', 'float', 'long']] #Getting all the columns having integer, float, long data type variable


In [0]:
string_cols

Out[39]: ['iso_code', 'continent', 'location', 'tests_units']

In [0]:
numeric_cols

Out[40]: ['total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_

In [0]:
df_filled = df.fillna("Unknown", subset=string_cols)

In [0]:
from pyspark.sql.functions import mean

# Create dictionary with column: mean_value
mean_values = df.select([
    mean(c).alias(c) for c in numeric_cols
]).collect()[0].asDict()

# Fill nulls
df_filled = df_filled.fillna(mean_values)

In [0]:
parquet_path = "s3a://dj-databricks-pipeline/owid-covid-parquet"

df_filled.write.mode("overwrite").parquet(parquet_path)
